Initialize Google Earth Engine (GEE)

In [7]:
import os, ee, geemap

ee.Authenticate()
ee.Initialize()

# Setup map
Map = geemap.Map()

Create Area of Interest (Dixie Fire California)

In [8]:
# Dixie Fire approximate bounding box (Butte/Tehama/Plumas counties, CA)
aoi = ee.Geometry.Polygon(
    [[[-121.8, 39.5], [-121.8, 40.3], [-120.7, 40.3], [-120.7, 39.5]]]
)
Map.centerObject(aoi, 8)
Map.addLayer(aoi, {}, "Dixie Fire AOI")
Map


Map(center=[39.90052064128846, -121.25000000000004], controls=(WidgetControl(options=['position', 'transparent…

Load and Explore Each Dataset

In [9]:
# MODIS Burned Area
modis = ee.ImageCollection("MODIS/006/MCD64A1").filterDate("2021-07-01", "2021-10-01").select("BurnDate")
modis_mosaic = modis.mosaic().clip(aoi)

Map.addLayer(modis_mosaic, {"min": 1, "max": 366, "palette": ["yellow", "red"]}, "MODIS Burned Area")

# VIIRS Active Fires
viirs = ee.ImageCollection("NASA/LANCE/NOAA20_VIIRS/C2").filterDate("2021-07-01", "2021-10-01").select("FRP")
viirs_mosaic = viirs.mosaic().clip(aoi)

Map.addLayer(viirs_mosaic, {"min": 0, "max": 500, "palette": ["blue", "orange", "red"]}, "VIIRS FRP")

# Sentinel-2 NDVI
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

s2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate("2021-06-01", "2021-07-01") \
    .filterBounds(aoi) \
    .map(maskS2clouds)

ndvi = s2.mean().normalizedDifference(["B8", "B4"]).rename("NDVI")
Map.addLayer(ndvi, {"min": 0, "max": 1, "palette": ["white", "green"]}, "Sentinel-2 NDVI (Pre-Fire)")

# ESA WorldCover
landcover = ee.ImageCollection("ESA/WorldCover/v100").first().clip(aoi)
Map.addLayer(landcover, {}, "ESA Land Cover")

# ERA5 Land Climate (Temperature)
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2021-07-01", "2021-07-31") \
    .select("temperature_2m")

era5_mean_temp = era5.mean().clip(aoi)
Map.addLayer(era5_mean_temp, {"min": 290, "max": 310, "palette": ["blue", "yellow", "red"]}, "ERA5 Mean Temp (July 2021)")

# CHIRPS Precipitation
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
    .filterDate("2021-06-01", "2021-07-01") \
    .select("precipitation")

chirps_total = chirps.sum().clip(aoi)
Map.addLayer(chirps_total, {"min": 0, "max": 200, "palette": ["white", "blue"]}, "CHIRPS Precipitation (June 2021)")

# SRTM Digital Elevation Model
srtm = ee.Image("USGS/SRTMGL1_003").clip(aoi)
Map.addLayer(srtm, {"min": 0, "max": 3000, "palette": ["white", "black"]}, "SRTM DEM")

Map

Map(center=[39.90052064128846, -121.25000000000004], controls=(WidgetControl(options=['position', 'transparent…

Save Raw Data

In [14]:
def downsample_image(image, scale, crs='EPSG:4326'):
    return image.reproject(crs=crs, scale=scale)

ndvi_ds = downsample_image(ndvi, scale=30)
era5_ds = downsample_image(era5_mean_temp, scale=1000)
chirps_ds = downsample_image(chirps_total, scale=5000)

output_folder = "../data/raw"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
datasets = {
    "dixie_modis_burned_area": modis_mosaic,
    "dixie_viirs": viirs_mosaic,
    "dixie_ndvi_prefire": ndvi_ds,
    "dixie_esa_landcover": landcover,
    "dixie_era5_mean_temp": era5_ds,
    "dixie_chirps_precip": chirps_ds,
    "dixie_srtm_dem": srtm
}

for name, image in datasets.items():
    try:
        info = image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=aoi,
            scale=500,
            maxPixels=1e13
        ).getInfo()

        if info and all(v == 0 for v in info.values()):
            print(f"Skipping {name}: no data found in AOI.")
            continue

        out_path = os.path.join(output_folder, f"{name}.tif")
        scale = 50 if name not in ["dixie_modis_burned_area",
                                   "dixie_era5_mean_temp",
                                   "dixie_chirps_precip"] else (
                                   500 if name == "dixie_modis_burned_area" else
                                   1000 if name == "dixie_era5_mean_temp" else
                                   5000)

        print(f"Exporting {name} at scale {scale} m...")
        geemap.ee_export_image(
            image,
            filename=out_path,
            scale=scale,
            region=aoi,
            file_per_band=False
        )
        print(f"Saved {name} to {out_path}")
    except Exception as e:
        print(f"Error exporting {name}: {e}")

Exporting dixie_modis_burned_area at scale 500 m...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\johnm\Desktop\Comp-Sci-App-State\Capstone\Wildfire-Lifecycle\data\raw\dixie_modis_burned_area.tif
Saved dixie_modis_burned_area to ../data/raw\dixie_modis_burned_area.tif
Exporting dixie_viirs at scale 50 m...
Generating URL ...
An error occurred while downloading.
Expression evaluates to an image with no bands.
Saved dixie_viirs to ../data/raw\dixie_viirs.tif
Exporting dixie_ndvi_prefire at scale 50 m...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\johnm\Desktop\Comp-Sci-App-State\Capstone\Wildfire-Lifecycle\data\raw\dixie_ndvi_prefire.tif
Saved dixie_ndvi_prefire to ../data/raw\dixie_ndvi_prefire.tif
Exporting dixie_esa_landcover at scale 50 m...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\johnm\Desktop\Comp-Sci-App-State\Capstone\Wildfire-Lifecycle\data\raw\dixie_esa_landcover.tif
Saved dixie_esa_landcover to ../data/raw\dixie_esa_la